![scrna5/6](https://img.shields.io/badge/scrna5/6-lightgrey)
[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/scrna4.ipynb)
[![lamindata](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/record/core/Transform?uid=Qr1kIHvK506rz8)

# Train a machine learning model on a collection

Here, we iterate over the artifacts within a collection to train a machine learning model at scale.

In [ ]:
import lamindb as ln
import anndata as ad
import numpy as np

In [ ]:
ln.track()

Query our collection:

In [ ]:
collection = ln.Collection.filter(
    name="My versioned scRNA-seq collection", version="2"
).one()
collection.describe()

## Create a map-style dataset

Let us create a [map-style dataset](https://pytorch.org/docs/stable/data) using using {meth}`~lamindb.Collection.mapped`: a {class}`~lamindb.dev.MappedCollection`. This is what, e.g., the with the PyTorch `DataLoader` expects as an input.

Under-the-hood, it performs a virtual inner join of the features of the underlying `AnnData` objects and thus allows to work with very large collections.

You can either perform a virutal inner join:

In [ ]:
with collection.mapped(label_keys=["cell_type"], join="inner") as dataset:
    print(dataset.var_joint)

Or a virtual outer join:

In [ ]:
dataset = collection.mapped(label_keys=["cell_type"], join="outer")

In [ ]:
len(dataset.var_joint)

This is compatible with pytorch `DataLoader` because it implements `__getitem__` over a list of backed `AnnData` objects. This is how you access the 5th cell in the collection:

In [ ]:
dataset[5]

The `labels` are encoded into integers:

In [ ]:
dataset.encoders

## Create a pytorch DataLoader

Let us use a weighted sampler:

In [ ]:
from torch.utils.data import DataLoader, WeightedRandomSampler

# label_key for weight doesn't have to be in labels on init
sampler = WeightedRandomSampler(
    weights=dataset.get_label_weights("cell_type"), num_samples=len(dataset)
)
dataloader = DataLoader(dataset, batch_size=128, sampler=sampler)

We can now iterate through the data loader:

In [ ]:
for batch in dataloader:
    pass

Close the connections in {class}`~lamindb.dev.MappedCollection`:

In [ ]:
dataset.close()

:::{dropdown} In practice, use a context manager

```
with collection.mapped(label_keys=["cell_type"]) as dataset:
    sampler = WeightedRandomSampler(
        weights=dataset.get_label_weights("cell_type"), num_samples=len(dataset)
    )
    dataloader = DataLoader(dataset, batch_size=128, sampler=sampler)
    for batch in dataloader:
        pass
```
:::